In [1]:
from google.colab import drive
import pandas as pd        
drive.mount('/content/drive')

path = "/content/drive/My Drive/creditcard.csv"
data = pd.read_csv(path)
data.head()

Mounted at /content/drive


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


<p dir="rtl"> در این قسمت درصد داده های non_fraud مشخص شده<p/>

In [2]:
pr_non_fraud =(data['Class'].count()-data['Class'].sum())/data['Class'].count()
pr_non_fraud

0.9982725143693799

<p dir="rtl"> در این قسمت درصد داده های fraud محاسبه شده<p/>

In [3]:
pr_fraud =(data['Class'].sum())/data['Class'].count()
pr_fraud

0.001727485630620034

<p dir="rtl"> در مرحله پیش پردازش داده ها ستون Time حذف شده زیرا تاثیر زیادی در خروجی ندارد و بقیه ستون ها با استفاده از quantile به ۲۰ قسمت تقسیم شده اند و هر قسمت در ستون مجزا است که بر اساس محدود داده مقدار آن ۰ یا ۱ میتواند باشد.
جدول اصل حاوی ۵۸۱ ستون میباشد که یک ستون آن class است و  نوع آن را مشخص میکند.<p/>

In [4]:
from sklearn.preprocessing import KBinsDiscretizer

# Perform quantile binning on the 'Amount' feature
amount_discretizer = KBinsDiscretizer(n_bins=20, encode='onehot-dense', strategy='quantile')
amount_binned = amount_discretizer.fit_transform(data[['Amount']])

# Perform quantile binning on the 'V1' to 'V28' features
v_discretizer = KBinsDiscretizer(n_bins=20, encode='onehot-dense', strategy='quantile')
v_binned = v_discretizer.fit_transform(data.iloc[:, 1:29])

# Convert the result to DataFrames and add column names
amount_binned = pd.DataFrame(amount_binned, columns=[f'Amount_{i+1}' for i in range(20)])
v_binned = pd.DataFrame(v_binned, columns=[f'V{i+1}_{j+1}' for i in range(28) for j in range(20)])

# Concatenate the binned features with the original data
data = pd.concat([data, amount_binned, v_binned], axis=1)

for col in ['Amount','Time'] + ['V'+str(i) for i in range(1, 29)]:
   data = data.drop([col], axis=1)

data.head()

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_discretization.py:216: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_discretization.py:216: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


,Class,Amount_1,Amount_2,Amount_3,Amount_4,Amount_5,Amount_6,Amount_7,Amount_8,Amount_9,...,V28_11,V28_12,V28_13,V28_14,V28_15,V28_16,V28_17,V28_18,V28_19,V28_20
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


<p dir="rtl">برای گسسته سازی داده ها از quantile استفاده شده که تعداد در هر قسمت برابر است پس محاسبه نکردن آیتم پرتکرار ۲ تایی مشکلی ایجاد نمیکند.</p>

In [6]:
df =pd.DataFrame(columns= ['val1', 'val2','count' ,'support_fraud', 'support_non_fraud','confidence_non_fraud'])
df

,val1,val2,count,support_fraud,support_non_fraud,confidence_non_fraud


<p dir="rtl"> جدول df برای قوانین سه تایی کاربرد دارد <p/>
<p dir="rtl">مقدار count نشان دهنده تعداد  دفعاتی است که ستون val1, val2 با یکدیگر ۱ هستند<p/>
<p dir="rtl"> مقدار support_fraud تعداد دفعات است که ۱ بودن ستون val1,val2 باعث ایجاد داده fraud شده است<p/>
<p dir="rtl"> مقدار support_non_fraud تعداد دفعات است که ۱ بودن ستون val1,val2 باعث ایجاد داده non_fraud شده است<p/>

In [7]:
cols = data.drop(['Class'], axis=1).columns
i=0

for num1,col1 in enumerate(cols):   
      for num2,col2 in enumerate(cols):
        if num1<num2 and col1.split('_')[0]!=col2.split('_')[0]:
          counts_twice =(data[col1] * data[col2]).sum()
          if counts_twice>0 :
             counts_frad = (data[col1] * data[col2] * data['Class']).sum()
             df.loc[i]=[col1, col2, counts_twice, counts_frad, counts_twice-counts_frad, (counts_twice-counts_frad)/counts_twice]
             i+=1 
df.head()  

,val1,val2,count,support_fraud,support_non_fraud,confidence_non_fraud
0,Amount_1,V1_1,1074.0,18.0,1056.0,0.983240
1,Amount_1,V1_2,807.0,6.0,801.0,0.992565
2,Amount_1,V1_3,651.0,2.0,649.0,0.996928
3,Amount_1,V1_4,627.0,4.0,623.0,0.993620
4,Amount_1,V1_5,600.0,6.0,594.0,0.990000


In [8]:
df.to_csv('three_rule.csv')

In [71]:
df = pd.read_csv('three_rule.csv')

In [72]:
df.describe()

,Unnamed: 0,count,support_fraud,support_non_fraud,confidence_non_fraud
count,162310.000000,162310.000000,162310.000000,162310.000000,162310.000000
mean,81154.500000,712.412310,1.230682,711.181628,0.998194
std,46855.005433,270.948076,8.781583,269.857587,0.012483
min,0.000000,1.000000,0.000000,0.000000,0.000000
25%,40577.250000,583.000000,0.000000,582.000000,0.998650
50%,81154.500000,695.000000,0.000000,694.000000,1.000000
75%,121731.750000,806.000000,1.000000,805.000000,1.000000
max,162309.000000,8137.000000,406.000000,8129.000000,1.000000


# Fraud

<p dir="rtl"> ایجاد فیلتر برای استخراج قوانین fraud<p/>

In [73]:
df_fraud = df[(df['count'] >=500.0) & (df['count'] <=1000.0)& (df['support_fraud']>30.0) ]
df_fraud

,Unnamed: 0,val1,val2,count,support_fraud,support_non_fraud,confidence_non_fraud
120,120,Amount_1,V7_1,691.0,45.0,646.0,0.934877
180,180,Amount_1,V10_1,888.0,55.0,833.0,0.938063
220,220,Amount_1,V12_1,710.0,52.0,658.0,0.926761
419,419,Amount_1,V21_20,507.0,33.0,474.0,0.934911
1199,1199,Amount_3,V4_20,777.0,100.0,677.0,0.871300
...,...,...,...,...,...,...,...
150809,150809,V20_19,V21_20,915.0,41.0,874.0,0.955191
153821,153821,V21_20,V24_12,783.0,34.0,749.0,0.956577
158529,158529,V24_3,V27_20,660.0,31.0,629.0,0.953030
158549,158549,V24_3,V28_20,646.0,32.0,614.0,0.950464


In [74]:
df_fraud['confidence_fraud']=df_fraud["support_fraud"]/df_fraud["count"]
df_fraud['interest_fraud']= df_fraud['confidence_fraud']-pr_fraud
df_fraud['interest_fraud'] = df_fraud['interest_fraud'].abs()
df_fraud= df_fraud.drop(columns=['support_non_fraud', 'confidence_non_fraud'])
df_fraud

<ipython-input-74-b6cd292f7cf9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fraud['confidence_fraud']=df_fraud["support_fraud"]/df_fraud["count"]
<ipython-input-74-b6cd292f7cf9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fraud['interest_fraud']= df_fraud['confidence_fraud']-pr_fraud
<ipython-input-74-b6cd292f7cf9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,Unnamed: 0,val1,val2,count,support_fraud,confidence_fraud,interest_fraud
120,120,Amount_1,V7_1,691.0,45.0,0.065123,0.063396
180,180,Amount_1,V10_1,888.0,55.0,0.061937,0.060209
220,220,Amount_1,V12_1,710.0,52.0,0.073239,0.071512
419,419,Amount_1,V21_20,507.0,33.0,0.065089,0.063361
1199,1199,Amount_3,V4_20,777.0,100.0,0.128700,0.126973
...,...,...,...,...,...,...,...
150809,150809,V20_19,V21_20,915.0,41.0,0.044809,0.043081
153821,153821,V21_20,V24_12,783.0,34.0,0.043423,0.041695
158529,158529,V24_3,V27_20,660.0,31.0,0.046970,0.045242
158549,158549,V24_3,V28_20,646.0,32.0,0.049536,0.047808


<p dir="rtl"> پنج قانون برتر سه تایی برای داده fraud با معیار confidence و interest<p/>

In [75]:
df_fraud.sort_values( by=["confidence_fraud","interest_fraud"],ascending=False)[:5]

,Unnamed: 0,val1,val2,count,support_fraud,confidence_fraud,interest_fraud
107990,107990,V12_1,V17_1,515.0,370.0,0.718447,0.716719
131110,131110,V16_1,V17_1,614.0,356.0,0.579805,0.578077
107930,107930,V12_1,V14_1,715.0,393.0,0.549650,0.547923
51530,51530,V4_20,V10_1,584.0,318.0,0.544521,0.542793
70090,70090,V7_1,V17_1,608.0,330.0,0.542763,0.541036


استخراج قوانین ۴ تایی از روی قوانین ۳ تایی

In [76]:
df_fraud4 =pd.DataFrame(columns= ['val1', 'val2','val3','count' ,'support_fraud','confidence_fraud','interest_fraud'])
df_fraud4

,val1,val2,val3,count,support_fraud,confidence_fraud,interest_fraud


In [78]:
cols = df_fraud[['val1','val2']].values
dict_fraud = {}
i=0
for  col in cols:
    for col1 in cols:
      if len(set(col).intersection(set(col1)))==1: 
        if (col1[0] not in col) and (col[0].split('_')[0]!=col1[0].split('_')[0]) and (col[1].split('_')[0]!=col1[0].split('_')[0]) and not(frozenset({col[0],col[1],col1[0]}) in dict_fraud): 
              condition1 = (data[col[0]]* data[col[1]]* data[col1[0]]).sum()
              dict_fraud[frozenset({col[0],col[1],col1[0]})] = condition1
              if condition1>0:
                  condition = (data[col[0]]* data[col[1]]* data[col1[0]]*data['Class']).sum()
                  df_fraud4.loc[i]=[col[0], col[1], col1[0],condition1, condition,(condition/condition1), (condition/condition1) - pr_fraud]
                  i=i+1

        elif (col1[1] not in col) and (col[0].split('_')[0]!=col1[1].split('_')[0]) and (col[1].split('_')[0]!=col1[1].split('_')[0]) and not(frozenset({col[0],col[1],col1[1]}) in dict_fraud):
              condition1 = (data[col[0]]* data[col[1]]* data[col1[1]]).sum()
              dict_fraud[frozenset({col[0],col[1],col1[1]})] = condition1
              if condition1>0:
                  condition = (data[col[0]]* data[col[1]]* data[col1[1]]*data['Class']).sum()
                  df_fraud4.loc[i]=[col[0], col[1], col1[1],condition1, condition,(condition/condition1), (condition/condition1) - pr_fraud]
                  i=i+1

In [80]:
df_fraud4["interest_fraud"]=df_fraud4["interest_fraud"].abs()
df_fraud4

,val1,val2,val3,count,support_fraud,confidence_fraud,interest_fraud
0,Amount_1,V7_1,V10_1,180.0,42.0,0.233333,0.231606
1,Amount_1,V7_1,V12_1,58.0,43.0,0.741379,0.739652
2,Amount_1,V7_1,V21_20,180.0,30.0,0.166667,0.164939
3,Amount_1,V7_1,V4_19,37.0,5.0,0.135135,0.133408
4,Amount_1,V7_1,V11_20,72.0,41.0,0.569444,0.567717
...,...,...,...,...,...,...,...
3329,V19_19,V21_20,V20_19,52.0,1.0,0.019231,0.017503
3330,V19_19,V21_20,V24_12,48.0,9.0,0.187500,0.185773
3331,V20_19,V21_20,V24_12,59.0,4.0,0.067797,0.066069
3332,V24_3,V27_20,V28_20,238.0,23.0,0.096639,0.094911


In [81]:
df_fraud4.describe()

,count,support_fraud,confidence_fraud,interest_fraud
count,3334.000000,3334.000000,3334.000000,3334.000000
mean,92.222555,36.475405,0.332277,0.330666
std,82.241284,54.965804,0.245244,0.245086
min,2.000000,0.000000,0.000000,0.001727
25%,40.000000,5.000000,0.111275,0.109547
50%,61.000000,22.000000,0.291667,0.289939
75%,110.000000,37.000000,0.528571,0.526844
max,532.000000,365.000000,0.934783,0.933055


ایجاد فیلتر روی قوانین ۴ تایی

In [82]:
df_fraud4_filter = df_fraud4[(df_fraud4['count'] >=110) & (df_fraud4['support_fraud']>40.0) ]
df_fraud4_filter

,val1,val2,val3,count,support_fraud,confidence_fraud,interest_fraud
0,Amount_1,V7_1,V10_1,180.0,42.0,0.233333,0.231606
20,Amount_1,V10_1,V9_1,117.0,48.0,0.410256,0.408529
81,Amount_3,V4_20,V12_1,158.0,98.0,0.620253,0.618526
87,Amount_3,V4_20,V3_1,139.0,96.0,0.690647,0.688920
89,Amount_3,V4_20,V6_1,116.0,69.0,0.594828,0.593100
...,...,...,...,...,...,...,...
3284,V18_1,V19_20,V28_20,138.0,45.0,0.326087,0.324359
3304,V18_1,V20_20,V27_20,156.0,66.0,0.423077,0.421349
3306,V18_1,V20_20,V28_20,202.0,52.0,0.257426,0.255698
3322,V18_1,V27_1,V28_1,357.0,42.0,0.117647,0.115920


<p dis="rtl"> استجراج و مشاهد ۵ قاونون برتر چهار تایی در داده fraud <p/>

In [83]:
df_fraud4_filter.sort_values( by=["confidence_fraud","interest_fraud"],ascending=False)[:5]

,val1,val2,val3,count,support_fraud,confidence_fraud,interest_fraud
1672,V7_1,V12_1,V14_1,394.0,333.0,0.845178,0.843450
1264,V4_20,V10_1,V17_1,342.0,288.0,0.842105,0.840378
2202,V10_1,V17_1,V11_20,406.0,341.0,0.839901,0.838174
2204,V10_1,V17_1,V16_1,415.0,348.0,0.838554,0.836827
2203,V10_1,V17_1,V12_1,431.0,361.0,0.837587,0.835860


In [84]:
df_fraud4_filter.sort_values( by="support_fraud",ascending=False)[:5]

,val1,val2,val3,count,support_fraud,confidence_fraud,interest_fraud
2579,V12_1,V14_1,V17_1,458.0,365.0,0.796943,0.795216
2203,V10_1,V17_1,V12_1,431.0,361.0,0.837587,0.835860
2631,V12_1,V16_1,V17_1,440.0,355.0,0.806818,0.805091
2578,V12_1,V14_1,V16_1,450.0,352.0,0.782222,0.780495
2204,V10_1,V17_1,V16_1,415.0,348.0,0.838554,0.836827


# Non_fraud

 ایجاد فیلتر برای استخراج قوانین non_fraud

In [85]:
df_non_fraud = df[(df['support_non_fraud'] >=3000.0) & (df['confidence_non_fraud']>0.9)]
df_non_fraud['interest_non_frad']= df_non_fraud['confidence_non_fraud']-pr_non_fraud
df_non_fraud['interest_non_frad'] = df_non_fraud['interest_non_frad'].abs()
df_non_fraud= df_non_fraud.drop(columns=['support_fraud'])
df_non_fraud

<ipython-input-85-1e161ea46127>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_fraud['interest_non_frad']= df_non_fraud['confidence_non_fraud']-pr_non_fraud
<ipython-input-85-1e161ea46127>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_fraud['interest_non_frad'] = df_non_fraud['interest_non_frad'].abs()


,Unnamed: 0,val1,val2,count,support_non_fraud,confidence_non_fraud,interest_non_frad
10091,10091,Amount_19,V1_12,3367.0,3367.0,1.000000,0.001727
10100,10100,Amount_19,V2_2,3173.0,3172.0,0.999685,0.001412
10477,10477,Amount_19,V20_19,3303.0,3298.0,0.998486,0.000214
10656,10656,Amount_20,V2_1,8137.0,8129.0,0.999017,0.000744
10716,10716,Amount_20,V5_1,5411.0,5382.0,0.994641,0.003632
...,...,...,...,...,...,...,...
159969,159969,V25_1,V28_20,3119.0,3104.0,0.995191,0.003082
161910,161910,V27_1,V28_1,5766.0,5722.0,0.992369,0.005903
161931,161931,V27_2,V28_2,3474.0,3471.0,0.999136,0.000864
162265,162265,V27_18,V28_16,4700.0,4700.0,1.000000,0.001727


<p dir="rtl"> پنج قانون برتر سه تایی برای داده non_fraud با معیار confidence و interest<p/>

In [86]:
df_non_fraud.sort_values( by=["confidence_non_fraud","interest_non_frad"],ascending=False)[:5]

,Unnamed: 0,val1,val2,count,support_non_fraud,confidence_non_fraud,interest_non_frad
10091,10091,Amount_19,V1_12,3367.0,3367.0,1.0,0.001727
17113,17113,V1_11,V27_18,3461.0,3461.0,1.0,0.001727
17669,17669,V1_12,V28_14,3786.0,3786.0,1.0,0.001727
19284,19284,V1_16,V2_3,3043.0,3043.0,1.0,0.001727
20337,20337,V1_17,V28_7,3089.0,3089.0,1.0,0.001727


In [87]:
df_non_fraud.sort_values( by=["support_non_fraud"],ascending=False)[:5]

,Unnamed: 0,val1,val2,count,support_non_fraud,confidence_non_fraud,interest_non_frad
10656,10656,Amount_20,V2_1,8137.0,8129.0,0.999017,0.000744
11035,11035,Amount_20,V20_20,7368.0,7346.0,0.997014,0.001258
69829,69829,V6_20,V24_20,7029.0,7029.0,1.000000,0.001727
22269,22269,V2_1,V20_20,7011.0,7001.0,0.998574,0.000301
107949,107949,V12_1,V14_20,6631.0,6629.0,0.999698,0.001426


استخراج قوانین ۴ تایی از روی قوانین ۳ تایی

In [88]:
df_non_fraud4 =pd.DataFrame(columns= ['val1', 'val2','val3','count' ,'support_non_fraud','confidence_non_fraud','interest_non_fraud'])
df_non_fraud4

,val1,val2,val3,count,support_non_fraud,confidence_non_fraud,interest_non_fraud


In [89]:
cols = df_non_fraud[['val1','val2']].values
dict_non_fraud = {}
i=0
for  col in cols:
    for col1 in cols:
      if len(set(col).intersection(set(col1)))==1: 
        if (col1[0] not in col) and (col[0].split('_')[0]!=col1[0].split('_')[0]) and (col[1].split('_')[0]!=col1[0].split('_')[0]) and not(frozenset({col[0],col[1],col1[0]}) in dict_non_fraud): 
              condition1 = (data[col[0]]* data[col[1]]* data[col1[0]]).sum()
              dict_non_fraud[frozenset({col[0],col[1],col1[0]})] = condition1
              if condition1>0:
                  condition = (data[col[0]]* data[col[1]]* data[col1[0]]*data['Class']).sum()
                  df_non_fraud4.loc[i]=[col[0], col[1], col1[0],condition1, condition1-condition,(condition1-condition)/condition1, ((condition1-condition)/condition1) - pr_non_fraud]
                  i=i+1

        elif (col1[1] not in col) and (col[0].split('_')[0]!=col1[1].split('_')[0]) and (col[1].split('_')[0]!=col1[1].split('_')[0]) and not(frozenset({col[0],col[1],col1[1]}) in dict_non_fraud):
              condition1 = (data[col[0]]* data[col[1]]* data[col1[1]]).sum()
              dict_non_fraud[frozenset({col[0],col[1],col1[1]})] = condition1
              if condition1>0:
                  condition = (data[col[0]]* data[col[1]]* data[col1[1]]*data['Class']).sum()
                  df_non_fraud4.loc[i]=[col[0], col[1], col1[1],condition1, condition1-condition,(condition1-condition)/condition1, ((condition1-condition)/condition1) - pr_non_fraud]
                  i=i+1

In [90]:
df_non_fraud4["interest_non_fraud"] = df_non_fraud4["interest_non_fraud"].abs()
df_non_fraud4

,val1,val2,val3,count,support_non_fraud,confidence_non_fraud,interest_non_fraud
0,Amount_19,V1_12,V2_2,1041.0,1041.0,1.000000,0.001727
1,Amount_19,V1_12,V20_19,999.0,999.0,1.000000,0.001727
2,Amount_19,V1_12,V28_14,1033.0,1033.0,1.000000,0.001727
3,Amount_19,V2_2,V20_19,940.0,940.0,1.000000,0.001727
4,Amount_19,V2_2,V7_2,251.0,251.0,1.000000,0.001727
...,...,...,...,...,...,...,...
443,V20_20,V23_20,V28_20,992.0,986.0,0.993952,0.004321
444,V20_20,V28_20,V25_1,500.0,496.0,0.992000,0.006273
445,V20_20,V28_20,V27_20,1477.0,1419.0,0.960731,0.037541
446,V21_20,V23_1,V25_20,451.0,427.0,0.946785,0.051488


In [91]:
df_non_fraud4.describe()

,count,support_non_fraud,confidence_non_fraud,interest_non_fraud
count,448.000000,448.000000,448.000000,448.000000
mean,1294.162946,1270.491071,0.979520,0.019843
std,735.424553,728.413992,0.034248,0.033627
min,16.000000,14.000000,0.672727,0.000018
25%,823.250000,795.000000,0.972163,0.001727
50%,1198.500000,1167.000000,0.995046,0.003226
75%,1678.000000,1645.750000,1.000000,0.026110
max,5690.000000,5684.000000,1.000000,0.325545


ایجاد فیلتر روی قوانین ۴ تایی

In [93]:
df_non_fraud4_filter = df_non_fraud4[(df_non_fraud4['support_non_fraud'] >=2000.0) & (df_non_fraud4['confidence_non_fraud']>0.9)]
df_non_fraud4_filter

,val1,val2,val3,count,support_non_fraud,confidence_non_fraud,interest_non_fraud
5,Amount_20,V2_1,V5_1,3349.0,3345.0,0.998806,0.000533
6,Amount_20,V2_1,V7_20,2468.0,2466.0,0.999190,0.000917
7,Amount_20,V2_1,V20_20,5690.0,5684.0,0.998946,0.000673
8,Amount_20,V2_1,V21_20,3216.0,3209.0,0.997823,0.000449
9,Amount_20,V2_1,V23_1,4258.0,4251.0,0.998356,0.000084
...,...,...,...,...,...,...,...
389,V8_1,V20_1,V21_20,2765.0,2748.0,0.993852,0.004421
395,V8_1,V21_1,V20_20,2145.0,2120.0,0.988345,0.009928
407,V9_20,V10_20,V20_20,2380.0,2380.0,1.000000,0.001727
413,V10_20,V20_20,V27_20,2106.0,2106.0,1.000000,0.001727


<p dir="rtl"> پنج قانون برتر جهار تایی برای داده non_fraud با معیار confidence و interest<p/>

In [94]:
df_non_fraud4_filter.sort_values( by=["confidence_non_fraud","interest_non_fraud"],ascending=False)[:5]

,val1,val2,val3,count,support_non_fraud,confidence_non_fraud,interest_non_fraud
66,V1_1,V2_20,V10_20,2839.0,2839.0,1.0,0.001727
155,V1_1,V9_20,V10_20,2667.0,2667.0,1.0,0.001727
170,V1_1,V10_20,V27_20,2046.0,2046.0,1.0,0.001727
229,V1_19,V23_18,V26_15,2352.0,2352.0,1.0,0.001727
233,V1_19,V25_6,V26_15,2319.0,2319.0,1.0,0.001727


In [95]:
df_non_fraud4_filter.sort_values( by=["support_non_fraud"],ascending=False)[:5]

,val1,val2,val3,count,support_non_fraud,confidence_non_fraud,interest_non_fraud
7,Amount_20,V2_1,V20_20,5690.0,5684.0,0.998946,0.000673
9,Amount_20,V2_1,V23_1,4258.0,4251.0,0.998356,0.000084
27,Amount_20,V20_20,V23_1,3966.0,3959.0,0.998235,0.000038
249,V2_1,V20_20,V21_20,3715.0,3705.0,0.997308,0.000964
250,V2_1,V20_20,V23_1,3506.0,3501.0,0.998574,0.000301
